# Getting Started

This example notebook briefly describes and demonstrates how to use each of the estimators included with statswag.

StatSWAG implements several statistical estimators that, given noisy categorical predictions (labels) from mulitple labelers for a set of data samples, estimate both the accuracy of each individual labeler and the true label of each data instance.  Potential applications include:

1. aggregating labels from crowdsourced workers
2. estimating the accuracies of an ensemble of classifiers when true labels are not available
3. estimating the accuracy of a single classifier using the labels from human experts (test and evaluation scenario)

This notebook will walk you through the basics of using the StatSWAG package. In it, you'll learn how to import the different estimators and use them on provided datasets. It is assumed that you have already downloaded and installed the `statswag` python package. If you haven't, navigate to the top-level where `requirements.txt` is located and run the following two commands:

```
pip install -r requirements.txt
pip install .
```

_Note: You may need to restart your Jupyter Notebook server if you are installing statswag after already launching this notebook._

In [1]:
# You can import them all at once
from statswag.estimators import *

#...or import them each by name
from statswag.estimators import MLEOneParameterPerLabeler, IWMV, MajorityVote, Spectral, Agreement

For demo purposes, we create a very simple dataset of labels.

In [2]:
# Create a very simple matrix of labels, rows are data instances and columns are labels from distinct labelers
import numpy as np
labels = [[0,1,1],
          [0,0,1],
          [1,1,1],
          [0,0,0],
          [1,1,1],
          [0,0,1],
          [1,1,1],
          [0,1,0],
          [0,0,0],
          [1,0,1]]
labels = np.asarray(labels)

The process is the same to use each estimator:
1. Instantiate the estimator
2. "Fit" the estimator to the labels

The fit method returns a dictionary with the following fields:
- `accuracies` - list of length M that contains the estimated expert accuracies.
- `labels` - is a list of length N that contains the best-guess label for each sample.
- `probs` - NxL matrix, where L is the number of classes.  The (i,j) entry of this matrix is the estimated probability that sample i belongs to class j.  These are not necessarily calibrated.
- `class_names` - classes observed by the estimator, and the ordering gives you the correct order to reference columns of `probs`.

_Note: Some of these fields may be empty depending on the estimator_

### Majority Vote
Majority Vote estimates labeler accuracy comparing, for each data sample, the most common label amongst all labelers to that of a single labeler.  There are two options for how to do this, controlled by the ``hold_out`` paramter.  If ``hold_out`` is set to ``True`` (the default), then a labeler's own predictions are not used in the computation of it's own accuarcy.  When ``hold_out`` is ``False`` they are.

Note that majority vote breaks ties randomly, so results can vary between runs.

In [3]:
# 1. Instantiate the estimator
mv = MajorityVote(hold_out=True)

# 2. "Fit" the estimator to the expert labels
result = mv.fit(labels)

In [4]:
result

{'accuracies': [0.6, 0.6, 0.5],
 'labels': [1, 0, 1, 0, 1, 0, 1, 0, 0, 1],
 'probs': None,
 'class_names': array([0, 1])}

### Agreement
This estimator approximates labeler accuracy using pairwise agreement rates between labelers. 

The estimator makes some strong assumptions, that when violated, cause the underlying computations to break down.  If this occurs, ``None`` may be output for one or more labelers.  We have observed that this scenario is common when some labelers disagree most of the time with other labelers.

In [5]:
agreement = Agreement()
result = agreement.fit(labels)
result

{'accuracies': array([0.6118034 , 0.51601958, 0.51601958]),
 'labels': array([1, 0, 1, 0, 1, 0, 1, 0, 0, 1]),
 'probs': None,
 'class_names': array([0, 1])}

### Iterative Weighted Majority Vote
Similar to majority vote, but iteratively updates estimates of labeler accuracy and uses these to compute a weighted majority vote.

In [6]:
iwmv = IWMV()
result = iwmv.fit(labels)
result

{'accuracies': array([0.9, 0.8, 0.8]),
 'labels': array([1, 0, 1, 0, 1, 0, 1, 0, 0, 1]),
 'probs': None,
 'class_names': array([0, 1])}

### Spectral
The spectral estimator estimates labeler accuracy by utilizing a relationship between a matrix whose off-diagonal elements are those of the labeler covariance matrix and the labeler class-conditional accuracies.  

The algorithm involves the computation of an eigenvector, the sign of which is not unique.  We can choose to resolve this in various ways, the choice is controlled by the ``signassumption`` parameter.  The default is to set `signassumption = half`, which assumes that at least half the entries should be positive (positive sign corresponds to labelers that are better than random).  The other option is to use `first` which assumes that the first expert is better than random.

In [7]:
spectral = Spectral(signassumption='half')
result = spectral.fit(labels)
result

{'accuracies': array([0.98309722, 0.7143649 , 0.73999833]),
 'labels': array([1, 0, 1, 0, 1, 0, 1, 0, 0, 1]),
 'probs': None,
 'class_names': array([0, 1])}

### Maximum Likelihood Estimation
This estimator uses expectation maximization to approximate the maximum likelihood estimates of labeler accuracy.  Since expectation maxmimization in an iterative algorithm, you can adjust the termination criterion.  Use ``tol`` to change the difference between sucessive likelihoods that will stop the algorithm and use ``max_iter`` to set the maximum number of iterations.  Defaults appear to work well in general and are 10^(-6) and 100, respectively.

This estimator is a bit different than the others.  It is randomly initialized and converges to a local optima, so it is a good idea to run it several times and select the accuracy estimates corresponding to the highest log-likelihood.

At the moment, we are limited to inferring one parameter (accuracy) per labeler.  Future versions may include the ability to model the class-conditional accuracies of each labeler (true positive and true negative rate in the binary case)

_Note: When there are two classes, MLE sometimes converges to solutions in which all experts are worse than random due to an issue with identifiability.  If this happens, we invert the accuracies (and predicted labels, etc.) as we generally assume that all experts are better than random._

In [8]:
MLE = MLEOneParameterPerLabeler()
ll_list = []
results_list = []
for i in range(5):
    results_list.append(MLE.fit(labels))
    ll_list.append(MLE.expert_models.log_likelihood(labels, results_list[i]['class_names']))
index = np.argmax(ll_list)
results_list[index]

{'accuracies': array([0.99999278, 0.70000299, 0.70000152]),
 'labels': array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1]),
 'probs': array([[9.99973790e-01, 2.62103071e-05],
        [9.99995186e-01, 4.81410412e-06],
        [1.98942726e-06, 9.99998011e-01],
        [9.99999116e-01, 8.84213890e-07],
        [1.98942726e-06, 9.99998011e-01],
        [9.99995186e-01, 4.81410412e-06],
        [1.98942726e-06, 9.99998011e-01],
        [9.99995186e-01, 4.81417140e-06],
        [9.99999116e-01, 8.84213890e-07],
        [1.08315385e-05, 9.99989168e-01]]),
 'class_names': array([0, 1])}